In [6]:
import pandas as pd

Read csv files

In [7]:
df_BTC_2017 = pd.read_csv("archive/BTC-2017min.csv")
df_BTC_2018 = pd.read_csv("archive/BTC-2018min.csv")
df_BTC_2019 = pd.read_csv("archive/BTC-2019min.csv")
df_BTC_2020 = pd.read_csv("archive/BTC-2020min.csv")
df_BTC_2021 = pd.read_csv("archive/BTC-2021min.csv")

Concat all datafremes

In [14]:
concat_BTC = pd.concat([df_BTC_2017, df_BTC_2018], axis=0)
concat_BTC = pd.concat([concat_BTC, df_BTC_2019], axis=0)
concat_BTC = pd.concat([concat_BTC, df_BTC_2020], axis=0)
concat_BTC = pd.concat([concat_BTC, df_BTC_2021], axis=0)

Sort BTC dataframe by date

In [97]:
concat_BTC["date"] = pd.to_datetime(concat_BTC["date"])
BTC_df = concat_BTC.sort_values(by="date")

BTC_df.head()

,unix,date,symbol,open,high,low,close,Volume BTC,Volume USD
525598,1483228860,2017-01-01 00:01:00,BTC/USD,966.34,966.34,966.34,966.34,7.610000,7353.847400
525597,1483228920,2017-01-01 00:02:00,BTC/USD,966.16,966.37,966.16,966.37,8.087376,7815.397864
525596,1483228980,2017-01-01 00:03:00,BTC/USD,966.37,966.37,966.37,966.37,0.000000,0.000000
525595,1483229040,2017-01-01 00:04:00,BTC/USD,966.37,966.37,966.37,966.37,0.000000,0.000000
525594,1483229100,2017-01-01 00:05:00,BTC/USD,966.43,966.43,966.43,966.43,0.107000,103.408010


Cleaning Data

In [19]:
BTC_df.isnull().sum()

unix          0
date          0
symbol        0
open          0
high          0
low           0
close         0
Volume BTC    0
Volume USD    0
dtype: int64

In [20]:
BTC_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2675301 entries, 525598 to 0
Data columns (total 9 columns):
 #   Column      Dtype         
---  ------      -----         
 0   unix        int64         
 1   date        datetime64[ns]
 2   symbol      object        
 3   open        float64       
 4   high        float64       
 5   low         float64       
 6   close       float64       
 7   Volume BTC  float64       
 8   Volume USD  float64       
dtypes: datetime64[ns](1), float64(6), int64(1), object(1)
memory usage: 204.1+ MB


Checking for Null Values

In [22]:
print("Null values:", BTC_df.isnull().values.sum())

Null values: 0


In [23]:
print("NaN values:", BTC_df.isnull().values.any())

NaN values: False


In [ ]:
#criar as funções que limpam os dados mesmo assim 

Exploratory Data Analysis

In [99]:
#print the start date and the end  date

start_date = BTC_df.iloc[0][1]
end_date = BTC_df.iloc[-1][1]

print("Start date:", start_date)
print("End date:", end_date)

Start date: 2017-01-01 00:01:00
End date: 2022-03-01 03:43:00


In [102]:
BTC_df_date = BTC_df.copy()
BTC_df_date["date"] = BTC_df_date["date"].dt.date

BTC_df_date.head()

,unix,date,symbol,open,high,low,close,Volume BTC,Volume USD
525598,1483228860,2017-01-01,BTC/USD,966.34,966.34,966.34,966.34,7.610000,7353.847400
525597,1483228920,2017-01-01,BTC/USD,966.16,966.37,966.16,966.37,8.087376,7815.397864
525596,1483228980,2017-01-01,BTC/USD,966.37,966.37,966.37,966.37,0.000000,0.000000
525595,1483229040,2017-01-01,BTC/USD,966.37,966.37,966.37,966.37,0.000000,0.000000
525594,1483229100,2017-01-01,BTC/USD,966.43,966.43,966.43,966.43,0.107000,103.408010


In [104]:
BTC_df_date.groupby("date").agg({
    "open": ['min', 'max', 'mean'],
})

open                        
                 min       max          mean
date                                        
2017-01-01    960.53   1004.87    977.256602
2017-01-02    992.69   1032.00   1012.267604
2017-01-03   1001.00   1038.02   1020.001535
2017-01-04   1031.90   1139.70   1076.558840
2017-01-05    889.00   1136.72   1043.608646
...              ...       ...           ...
2022-02-25  38059.69  39678.17  38839.029090
2022-02-26  38666.63  40089.12  39234.306611
2022-02-27  37124.31  39827.93  38719.760201
2022-02-28  37472.27  43901.10  39415.151931
2022-03-01  42918.62  43621.64  43266.107040

[1860 rows x 3 columns]